In [1]:
import ast
import copy
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from aocd import get_data, submit

DAY = 18
YEAR = 2022

In [2]:
# use test data
raw_test = """2,2,2
1,2,2
3,2,2
2,1,2
2,3,2
2,2,1
2,2,3
2,2,4
2,2,6
1,2,5
3,2,5
2,1,5
2,3,5"""

# use real data
raw = get_data(day=DAY, year=YEAR)

# add comment


print(raw_test)

2,2,2
1,2,2
3,2,2
2,1,2
2,3,2
2,2,1
2,2,3
2,2,4
2,2,6
1,2,5
3,2,5
2,1,5
2,3,5


In [3]:
def parse_data(data):
    data = data.split("\n")
    data = np.array([re.findall("\d+", d) for d in data], dtype=int)
    data = set(map(tuple, data))

    return data


dummy = parse_data(raw_test)
real = parse_data(raw)

dummy

{(1, 2, 2),
 (1, 2, 5),
 (2, 1, 2),
 (2, 1, 5),
 (2, 2, 1),
 (2, 2, 2),
 (2, 2, 3),
 (2, 2, 4),
 (2, 2, 6),
 (2, 3, 2),
 (2, 3, 5),
 (3, 2, 2),
 (3, 2, 5)}

# Part 1

In [60]:
data = real.copy()

directions = [
    [1, 0, 0],
    [-1, 0, 0],
    [0, 1, 0],
    [0, -1, 0],
    [0, 0, 1],
    [0, 0, -1],
]
directions = np.array(directions)


def surf_v1(data):
    obs_nb_dict = {tuple(pt): set(map(tuple, (pt + directions))) & data for pt in data}
    return sum([6 - len(nb) for nb in obs_nb_dict.values()])


result = surf_v1(data)
result

3662

In [5]:
# submit(result, part="a", day=DAY, year=YEAR)

# Part 2

In [84]:
def is_unbound(air_pt, data):
    if air_pt in data:
        return False

    ptx, pty, ptz = air_pt
    colx = {d[0] for d in data if d[1] == pty and d[2] == ptz}
    coly = {d[1] for d in data if d[0] == ptx and d[2] == ptz}
    colz = {d[2] for d in data if d[0] == ptx and d[1] == pty}

    free_x = len(colx) == 0 or (ptx > max(colx) or ptx < min(colx))
    free_y = len(coly) == 0 or (pty > max(coly) or pty < min(coly))
    free_z = len(colz) == 0 or (ptz > max(colz) or ptz < min(colz))

    return any([free_x, free_y, free_z])

In [129]:
data = real.copy()

# same as before
obs_nb_dict = {tuple(pt): set(map(tuple, (pt + directions))) & data for pt in data}

# all neighboring air (air surrounded by air is missed)
air_nb_dict = {tuple(pt): set(map(tuple, (pt + directions))) - data for pt in data}
pocket_cands = {pt for v in air_nb_dict.values() for pt in v if not is_unbound(pt, data)}
pocket_cands

# remove air blocks which eventually connect to unbound air blocks
not_pockets = set()
while True:
    for pt in pocket_cands:
        pt_air_nb = set(map(tuple, (pt + directions))) - data
        connected_to_unbound = {npt for npt in pt_air_nb if is_unbound(npt, data) or npt in not_pockets}
        if len(connected_to_unbound) > 0:
            not_pockets |= connected_to_unbound.union({pt})

    pockets = pocket_cands - not_pockets
    if pockets == pocket_cands:
        break

    pocket_cands = pockets

# "fill" air pockets
new_obs_nb_dict = {tuple(pt): obs_nb_dict[pt] | (set(map(tuple, (pt + directions))) & pocket_cands) for pt in data}
result = sum([6 - len(nb) for nb in new_obs_nb_dict.values()])
result

2060

In [130]:
# submit(result, part="b", day=DAY, year=YEAR)